In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import plotly.express as px
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df=pd.read_csv('/kaggle/input/global-data-on-sustainable-energy/global-data-on-sustainable-energy (1).csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
#df.drop(columns=['Financial flows to developing countries (US $)','Renewables (% equivalent primary energy)',] ,inplace = True)

In [ ]:
energy_land = df[['Entity', 'Land Area(Km2)']]#create new data 
energy_land = energy_land.dropna()
#===========================================
countries = energy_land['Entity'].unique()#take the unique value
#===========================================
land = energy_land['Land Area(Km2)'].unique()#take the unique value
land_int = []#use to save new values
for num in land:
    land_int.append(num.replace(',', ''))#remove ','
for num in land_int:#convert each value to int
    int(num)
#===========================================
energy_land_data_use_df = pd.DataFrame([countries, pd.to_numeric(land_int)], index=['Country', 'Land'])#create dataframe
energy_land_data_use_df = energy_land_data_use_df.T
#===========================================
fig = px.bar(energy_land_data_use_df, x='Country', y='Land', labels={'Land': 'Land Area - km2', 'Entity': 'Country'})#create relation 
# Updates graph layout/styles
fig.update_layout(title={'text': 'Countries Land Area - in km2','x': 0.5,})#for the title
# Shows the graph
fig.show()

# Getting the CO2 emission values from the 3 biggest countries¶


In [ ]:
energy_co2_data = df[['Entity', 'Year', 'Value_co2_emissions_kt_by_country']].rename(columns={'Value_co2_emissions_kt_by_country': 'co2'})#create new data 
#energy_co2_data=energy_co2_data.dropna()

In [ ]:
# Canada DataFrame with dropped missing values
energy_co2_data_canada = energy_co2_data[(energy_co2_data['Entity'] == 'Canada')].dropna()
# United States DataFrame with dropped missing values
energy_co2_data_united_states = energy_co2_data[(energy_co2_data['Entity'] == 'United States')].dropna()
# China DataFrame with dropped missing values
energy_co2_data_china = energy_co2_data[(energy_co2_data['Entity'] == 'China')].dropna()

In [ ]:
# Imports important packages from plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# Creates the subplot for the 3 graphs
fig = make_subplots( rows=2, cols=2,specs=[[{}, {}],[{"colspan": 2}, None]],subplot_titles=('Canada', 'United States', 'China'))
# Adds the 3 graphs into the subplot
fig.add_trace( go.Bar(x=energy_co2_data_canada['Year'], y=energy_co2_data_canada['co2']), row=1, col=1)
fig.add_trace( go.Bar(x=energy_co2_data_united_states['Year'], y=energy_co2_data_united_states['co2']), row=1, col=2)
fig.add_trace(go.Bar(x=energy_co2_data_china['Year'], y=energy_co2_data_china['co2']),row=2, col=1)
# Updates subplot layout
fig.update_layout(height=600, width=1000, showlegend=False, title={'text': 'CO2 emission - in kiloton - by the three biggest countries in the world, per year','x': 0.5})
# Shows subplot
fig.show()

# Getting 2019 CO2 emissions

In [ ]:
# Gets all the years of 2019
energy_co2_data_2019 = energy_co2_data[(energy_co2_data['Year'] == 2019)]
# Drops the missing values
energy_co2_data_2019 = energy_co2_data_2019.dropna()
# Shows it columns
energy_co2_data_2019.columns

In [ ]:
# Creates the graph of 2019 CO2 emissions
fig_co2_2019 = px.bar(energy_co2_data_2019, x='Entity', y='co2')
# Updates graph layout
fig_co2_2019.update_layout(title={'text': 'CO2 emission - in kiloton - by all the countries in the world, per year', 'x': 0.5})
# Shows graph
fig_co2_2019.show()

# MAP SHAPE

In [ ]:
# Function to plot features on world map
def plot_world_map(column_name):
    fig = go.Figure()
    for year in range(2000, 2021):
        # Filter the data for the current year
        filtered_df = df[df['Year'] == year]

        # Create a choropleth trace for the current year
        trace = go.Choropleth(
            locations=filtered_df['Entity'],
            z=filtered_df[column_name],
            locationmode='country names',
            colorscale='Electric',  # Use a different color scale for better contrast
            colorbar=dict(title=column_name),
            zmin=df[column_name].min(),
            zmax=df[column_name].max(),
            visible=False  # Set the trace to invisible initially
        )

        # Add the trace to the figure
        fig.add_trace(trace)

    # Set the first trace to visible
    fig.data[0].visible = True

    # Create animation steps
    steps = []
    for i in range(len(fig.data)):
        step = dict(
            method='update',
            args=[{'visible': [False] * len(fig.data)},  # Set all traces to invisible
                  {'title_text': f'{column_name} Map - {2000 + i}', 'frame': {'duration': 1000, 'redraw': True}}],
            label=str(2000 + i)  # Set the label for each step
        )
        step['args'][0]['visible'][i] = True  # Set the current trace to visible
        steps.append(step)

    # Create the slider
    sliders = [dict(
        active=0,
        steps=steps,
        currentvalue={"prefix": "Year: ", "font": {"size": 14}},  # Increase font size for slider label
    )]

    # Update the layout of the figure with increased size and change the template
    fig.update_layout(
        title_text=f'{column_name} Map with slider',  # Set the initial title
        title_font_size=24,  # Increase title font size
        title_x=0.5,  # Center the title
        geo=dict(
            showframe=True,
            showcoastlines=False,
            projection_type='natural earth'
        ),
        sliders=sliders,
        height=500,  # Set the height of the figure in pixels
        width=1000,  # Set the width of the figure in pixels
        font=dict(family='Arial', size=12),  # Customize font family and size for the whole figure
        margin=dict(t=80, l=50, r=50, b=50),  # Add margin for better layout spacing
        # Change the template to 'plotly_dark'
    )

    # Show the figure
    fig.show()

In [ ]:
#select_col=df.columns# لو استخدمت دي بيطلع ايرور 
select_col = [
# 'Access to electricity (% of population)',
 'Access to clean fuels for cooking',
 'Renewable energy share in the total final energy consumption (%)']

In [ ]:
for i in select_col:
    column_name = i
    print(column_name)
    plot_world_map(column_name)

In [ ]:
plt.figure(figsize=(45,45))
Corr_Matrix=df.corr()
sns.heatmap(Corr_Matrix,annot=True)

In [ ]:
df.isna().sum()

In [ ]:
#df.drop(columns=['Renewables (% equivalent primary energy)','Financial flows to developing countries (US $)','Primary energy consumption per capita (kWh/person)'] ,inplace = True)
#df=df.dropna()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]
# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]
# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

In [ ]:
print('Top 5 Most Positively Correlated to the Target Variable')
Corr_Matrix['Electricity from nuclear (TWh)'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Negatively Correlated to the Target Variable')
Corr_Matrix['Electricity from nuclear (TWh)'].sort_values(ascending=True).head(5)

In [ ]:
columns_to_drop = [col for col in Corr_Matrix.columns if abs(Corr_Matrix.loc['Electricity from nuclear (TWh)', col]) < 0.5]
columns_to_drop

In [ ]:
df = df.drop(columns_to_drop, axis=1)
df.shape

In [ ]:
df=df.dropna()

In [ ]:
df.dtypes

In [ ]:
df['Land Area(Km2)'] = df['Land Area(Km2)'].str.replace(',', '').astype(int) 

In [ ]:
df.rename(columns={'Density\\n(P/Km2)': 'New_Density'}, inplace=True)

In [ ]:
df['New_Density'] = df['New_Density'].str.replace(',', '').astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
df.Entity = le.fit_transform(df.Entity)


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
X = df.drop(columns=['Electricity from nuclear (TWh)'])
y = df['Electricity from nuclear (TWh)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual Price'] = y_test
    submit['Predict_price'] = y_pred
    submit = submit.reset_index()
    print(submit.head(8))
    r2 = r2_score(y_test, y_pred)

    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")

In [ ]:
importances = model.feature_importances_

feature_names = X.columns

feature_importance_dict = dict(zip(feature_names, importances))

sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.2f}")

plt.figure(figsize=(12, 7))
plt.barh(*zip(*sorted_feature_importance), alpha=0.9, color='teal')
plt.title('Feature Importance', fontsize=15)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()


In [ ]:
import pandas as pd
import statsmodels.api as sm

# Your DataFrame
# df = ...

X = df.drop(columns=['Electricity from nuclear (TWh)'])
y = df['Electricity from nuclear (TWh)']

def forward_selection(df, target, significance_level=0.05):
    initial_features = df.columns.tolist()
    best_features = []
    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(df[best_features + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < significance_level:
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

# Assuming you have already defined X and y as the features and target variable respectively
selected_features = forward_selection(X, y)
print("Selected features:", selected_features)


In [ ]:
Selected_features = [ 'Value_co2_emissions_kt_by_country', 'Land Area(Km2)', 'Entity', 'New_Density']

X = df[selected_features]
y = df['Electricity from fossil fuels (TWh)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual Price'] = y_test
    submit['Predict_price'] = y_pred
    submit = submit.reset_index()
    print(submit.head(8))
    r2 = r2_score(y_test, y_pred)

    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")